In [12]:
import os

from dotenv import load_dotenv

load_dotenv()

# OpenAI API Key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_URL = "https://api.openai.com/v1/chat/completions"

elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")
openai_api_key=os.getenv('OPENAI_API_KEY')

In [13]:
from PIL import Image
import requests
import numpy as np

from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

image_path = 'https://i.imgur.com/IxC9lgd.png'

chat = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=512, openai_api_key=openai_api_key)
result = chat.invoke(
    [
        HumanMessage(
            content=[
                {"type": "text", "text": "What is in the picture? Please provide a detailed introduction."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_path,
                        "detail": "auto",
                    },
                },
            ]
        )
    ]
)


print(result.content)

This image shows a handwritten letter to Santa Claus, presumably written by a child named Maria, who is 8 years old and lives in Rio de Janeiro. The letter is on a torn piece of lined notebook paper, indicating it may have been removed from a spiral-bound notebook.

In the letter, Maria politely inquires about Santa's well-being before introducing herself. She then states that she is writing to ask for a Christmas present. She emphasizes that she has been well-behaved throughout the year, mentioning that she has done her homework, helped her parents with chores, and been kind to her friends.

Maria lists her Christmas wishes, specifically asking for "Barbie Dreamhouse Adventures" and "My Little Pony." These items suggest that Maria enjoys popular children's toys and shows associated with those toys.

She closes the letter by acknowledging Santa's busy schedule and expresses hope that he will be able to bring her the requested presents. The letter concludes with a polite thank you and i

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)

prompt = PromptTemplate.from_template(
"""
Extract the list and child's name from the text below and return the data in JSON format using the following name:
- "child_name", "wishlist".

{santalist}

"""
)

runnable = prompt | chain | StrOutputParser()

letter = result.content
wishlist = runnable.invoke({"santalist": letter})
print(wishlist)

{
  "child_name": "Maria",
  "wishlist": [
    "Barbie Dreamhouse Adventures",
    "My Little Pony"
  ]
}


In [16]:
chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)

prompt = PromptTemplate.from_template(
"""

{santalist_json}

From the JSON above, include a new attribute in the JSON called 'weight',
which will calculate the total estimated weight of each item in the list in kilograms.

You will first need to estimate the weight of each item individually.
After that, sum these values to obtain the total weight.
Extract only the numerical value.

"""
)

runnable = prompt | chain | StrOutputParser()

new_wishlist = runnable.invoke({"santalist_json": wishlist})
print(new_wishlist)

{
  "child_name": "Maria",
  "wishlist": [
    "Barbie Dreamhouse Adventures",
    "My Little Pony"
  ],
  "weight": 0.5
}


In [17]:
from elasticsearch import Elasticsearch
import json

url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"
es = Elasticsearch(url, ca_certs = "./http_ca.crt", verify_certs = True)

es.info() # should return cluster info

# Parse the JSON string
json_string = new_wishlist
data = json.loads(json_string)

# Index name
index_name = "santa_claus_list"

# Index the document
response = es.index(index=index_name, document=data)

# Print the response from Elasticsearch
print(response)

{'_index': 'santa_claus_list', '_id': 'rHoY8owB6xtD7DLUdxmY', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
